In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages=True, memory_key="context")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key="context")

chain = (
    {
       
        "context": RunnableMap(
            {
                "retriever_context": retriver,
                "memory_context": lambda inputs: memory.load_memory_variables({}),
            }
        ) 
      
        | (lambda inputs: {"context": "\n".join(["\n".join([doc.page_content for doc in inputs["retriever_context"]]), 
            inputs["memory_context"].get("chat_history", "")
        ]
    )
}),
       
        "question": RunnablePassthrough(),
    }
   
    | prompt

    | llm
 
    | (lambda outputs: memory.save_context({"input": outputs.content}, {"output": outputs.content}) or outputs)
)

chain.invoke("Is Aaronson guilty?")

AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [21]:
chain.invoke("What message did he write in the table?")

AIMessage(content='He wrote "2+2=5" in the dust on the table.')

In [22]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character in the text who is mentioned as someone Winston loves and cares for deeply.')

In [23]:
for idx, entry in enumerate(memory.buffer):
    print(f"Entry {idx + 1}: content='{entry.content}' (Type: {type(entry)})")

Entry 1: content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' (Type: <class 'langchain.schema.messages.HumanMessage'>)
Entry 2: content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' (Type: <class 'langchain.schema.messages.AIMessage'>)
Entry 3: content='He wrote "2+2=5" in the dust on the table.' (Type: <class 'langchain.schema.messages.HumanMessage'>)
Entry 4: content='He wrote "2+2=5" in the dust on the table.' (Type: <class 'langchain.schema.messages.AIMessage'>)
Entry 5: content='Julia is a character in the text who is mentioned as someone Winston loves and cares for deeply.' (Type: <class 'langchain.schema.messages.HumanMessage'>)
Entry 6: content='Julia is a character in the text who is mentioned as someone Winston loves and cares for deeply.' (Type: <class 'langchain.schema.messages.AIMessage'>)
